# Universidad del Valle de Guatemala
## Facultad de Ingeniería
### Departamento de Computación

---

# Proyecto 2: Resultados Iniciales
### Reto 04: Predicción de compradores recurrentes: cuestionar la línea base - Negocios

**Integrantes:**
- Diego Alexander Hernández Silvestre, 21270
- Linda Inés Jiménez Vides, 21169
- Mario Antonio Guerra Morales, 21008
- David Jonathan Aragon Vasquez, 21053

**Curso:** Data Science  
**Sección:** 10  
**Grupo** 4  

---

Guatemala, 28 de octubre de 2024


##### 🫳 Para la descarga del dataset de este reto, es necesario la creación de una cuenta dentro de Alibaba Cloud. Para facilitar este proceso se proporcionan los elementos a descargar en estos enlaces:
- Data_Format1: https://drive.google.com/file/d/1jgsaSM0wypPMUWaNlXy5s_kO0uiXNzES/view?usp=sharing
- Data_Format2: https://drive.google.com/file/d/1dnMeaon-ovASN2HEI1WSNfnw-7lyINAh/view?usp=drivesdk
- sample_submission.csv: 

In [1]:
import os
import zipfile
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
import matplotlib.ticker as ticker
from imblearn.over_sampling import SMOTE

### 📩 Carga de los datos

In [2]:
dataTrain_timestamp = pd.read_csv('data/dataTrain_balanced_timestamp.csv')
dataTrain = pd.read_csv('data/dataTrain_balanced.csv')

In [3]:
dataTrain_timestamp.head()

,Unnamed: 0.1,user_id,age_range,gender,merchant_id,item_id,category_id,brand_id,clicks,add_to_cart,purchases,add_to_favorites,total_actions,unique_item_count,label,time_stamp
0,0,34176,6.0,0.0,944,408895,1505,7370,1,0,0,0,1,1,-1,"DatetimeIndex(['2023-11-07'], dtype='datetime6..."
1,1,34176,6.0,0.0,412,17235,1604,4396,8,0,0,0,8,7,-1,"DatetimeIndex(['2023-08-18', '2023-08-18', '20..."
2,2,34176,6.0,0.0,1945,231901,662,2758,7,0,0,0,7,3,-1,"DatetimeIndex(['2023-08-18', '2023-08-18', '20..."
3,3,34176,6.0,0.0,4752,174142,821,6938,1,0,0,0,1,1,-1,"DatetimeIndex(['2023-10-27'], dtype='datetime6..."
4,4,34176,6.0,0.0,643,716371,1505,968,0,0,0,1,1,1,-1,"DatetimeIndex(['2023-10-24'], dtype='datetime6..."


In [4]:
dataTrain.head()

,Unnamed: 0.1,user_id,age_range,gender,merchant_id,item_id,category_id,brand_id,clicks,add_to_cart,purchases,add_to_favorites,total_actions,unique_item_count,label
0,0,34176,6.0,0.0,944,408895,1505,7370,1,0,0,0,1,1,-1
1,1,34176,6.0,0.0,412,17235,1604,4396,8,0,0,0,8,7,-1
2,2,34176,6.0,0.0,1945,231901,662,2758,7,0,0,0,7,3,-1
3,3,34176,6.0,0.0,4752,174142,821,6938,1,0,0,0,1,1,-1
4,4,34176,6.0,0.0,643,716371,1505,968,0,0,0,1,1,1,-1


In [5]:
print("Data Train Timestamp\n")
dataTrain_timestamp.info()
dataTrain_timestamp.count()

print("\nData Train without Timestamp\n")
dataTrain.info()
dataTrain.count()

Data Train Timestamp

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20300811 entries, 0 to 20300810
Data columns (total 16 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Unnamed: 0.1       int64  
 1   user_id            int64  
 2   age_range          float64
 3   gender             float64
 4   merchant_id        int64  
 5   item_id            int64  
 6   category_id        int64  
 7   brand_id           int64  
 8   clicks             int64  
 9   add_to_cart        int64  
 10  purchases          int64  
 11  add_to_favorites   int64  
 12  total_actions      int64  
 13  unique_item_count  int64  
 14  label              int64  
 15  time_stamp         object 
dtypes: float64(2), int64(13), object(1)
memory usage: 2.4+ GB

Data Train without Timestamp

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20300811 entries, 0 to 20300810
Data columns (total 15 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Unnamed: 

Unnamed: 0.1         20300811
user_id              20300811
age_range            20300811
gender               20300811
merchant_id          20300811
item_id              20300811
category_id          20300811
brand_id             20300811
clicks               20300811
add_to_cart          20300811
purchases            20300811
add_to_favorites     20300811
total_actions        20300811
unique_item_count    20300811
label                20300811
dtype: int64

In [6]:
dataTrain.describe()

,Unnamed: 0.1,user_id,age_range,gender,merchant_id,item_id,category_id,brand_id,clicks,add_to_cart,purchases,add_to_favorites,total_actions,unique_item_count,label
count,2.030081e+07,2.030081e+07,2.030081e+07,2.030081e+07,2.030081e+07,2.030081e+07,2.030081e+07,2.030081e+07,2.030081e+07,2.030081e+07,2.030081e+07,2.030081e+07,2.030081e+07,2.030081e+07,2.030081e+07
mean,3.513824e+06,2.116874e+05,3.037811e+00,3.359498e-01,2.512599e+03,5.559079e+05,8.667681e+02,4.163890e+03,8.630119e+00,5.517021e-06,9.027854e-01,3.033002e-01,1.008873e+01,4.353554e+00,0.000000e+00
std,2.027742e+06,1.215602e+05,1.710790e+00,4.892762e-01,1.287225e+03,3.217158e+05,3.902042e+02,2.163676e+03,2.279894e+01,2.410922e-03,8.730307e-01,1.382527e+00,2.359851e+01,8.344923e+00,8.164966e-01
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,-1.000000e+00
25%,1.765179e+06,1.056220e+05,2.000000e+00,0.000000e+00,1.480000e+03,2.750960e+05,5.960000e+02,2.368000e+03,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.000000e+00,1.000000e+00,-1.000000e+00
50%,3.513432e+06,2.122770e+05,3.000000e+00,0.000000e+00,2.492000e+03,5.548880e+05,8.490000e+02,4.073000e+03,4.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,5.000000e+00,2.000000e+00,0.000000e+00
75%,5.261091e+06,3.166630e+05,4.000000e+00,7.078987e-01,3.596000e+03,8.307000e+05,1.180000e+03,5.910000e+03,9.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.100000e+01,4.000000e+00,1.000000e+00
max,7.030722e+06,4.241700e+05,8.000000e+00,2.000000e+00,4.995000e+03,1.113166e+06,1.671000e+03,8.477000e+03,6.962000e+03,2.000000e+00,2.020000e+02,3.980000e+02,6.963000e+03,1.194000e+03,1.000000e+00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Función para reducir el uso de memoria ajustando los tipos de datos
def reduce_mem_usage(df):
    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type).startswith('int'):
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)
            else:
                df[col] = df[col].astype(np.float32)

    return df

# Definir el tamaño del chunk
chunk_size = 100000  # Puedes ajustar este tamaño según tu memoria disponible

# Leer el dataset por fragmentos y procesarlo
data_chunks = pd.read_csv('data/dataTrain_balanced_timestamp.csv', chunksize=chunk_size, usecols=[
    'user_id', 'age_range', 'gender', 'merchant_id', 'item_id', 'category_id', 
    'brand_id', 'clicks', 'add_to_cart', 'purchases', 'add_to_favorites', 
    'total_actions', 'unique_item_count', 'label', 'time_stamp'
])

# Lista para almacenar los chunks procesados
data_list = []

for chunk in data_chunks:
    # Reducir el uso de memoria en cada chunk
    chunk = reduce_mem_usage(chunk)

    # Convertir la columna 'time_stamp' a tipo datetime
    chunk['time_stamp'] = pd.to_datetime(chunk['time_stamp'], errors='coerce')

    # Codificar las columnas categóricas con LabelEncoder
    le = LabelEncoder()
    chunk['gender'] = le.fit_transform(chunk['gender'].astype(str))
    chunk['age_range'] = le.fit_transform(chunk['age_range'].astype(str))

    # Agregar el chunk procesado a la lista
    data_list.append(chunk)

# Concatenar todos los chunks procesados
dataTrain_timestamp = pd.concat(data_list, axis=0)

# Definir características (X) y etiqueta (y)
X = dataTrain_timestamp.drop(columns=['label', 'time_stamp'])  # Excluir 'label' y 'time_stamp'
y = dataTrain_timestamp['label']

# Dividir el dataset en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar las características
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Instanciar el modelo de Regresión Logística
model = LogisticRegression(max_iter=1000)

# Entrenar el modelo
model.fit(X_train, y_train)

# Hacer predicciones en el conjunto de prueba
y_pred = model.predict(X_test)

# Imprimir resultados
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print('Confusion Matrix:')
print(confusion_matrix(y_test, y_pred))
print('Classification Report:')
print(classification_report(y_test, y_pred))

# Visualización de la matriz de confusión
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout
from tensorflow.keras.optimizers import Adam

# Función para reducir el uso de memoria ajustando los tipos de datos
def reduce_mem_usage(df):
    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type).startswith('int'):
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)
            else:
                df[col] = df[col].astype(np.float32)
    return df

# Leer los datos en chunks y reducir el uso de memoria
chunk_size = 100000
data_chunks = pd.read_csv('data/dataTrain_balanced_timestamp.csv', chunksize=chunk_size, usecols=[
    'user_id', 'age_range', 'gender', 'merchant_id', 'item_id', 'category_id', 
    'brand_id', 'clicks', 'add_to_cart', 'purchases', 'add_to_favorites', 
    'total_actions', 'unique_item_count', 'label', 'time_stamp'
])

data_list = []

for chunk in data_chunks:
    chunk = reduce_mem_usage(chunk)
    chunk['time_stamp'] = pd.to_datetime(chunk['time_stamp'], errors='coerce')
    le = LabelEncoder()
    chunk['gender'] = le.fit_transform(chunk['gender'].astype(str))
    chunk['age_range'] = le.fit_transform(chunk['age_range'].astype(str))
    data_list.append(chunk)

dataTrain_timestamp = pd.concat(data_list, axis=0)

# Selección de características para redes neuronales
dataTrain_timestamp = dataTrain_timestamp.sort_values(by=['user_id', 'time_stamp'])  # Ordenamos por usuario y tiempo
X = dataTrain_timestamp.drop(columns=['label', 'time_stamp'])
y = dataTrain_timestamp['label']

scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convertimos X e y en una secuencia de tiempo
sequence_length = 10  # Número de pasos de tiempo
X_seq, y_seq = [], []
for i in range(len(X) - sequence_length):
    X_seq.append(X[i:i + sequence_length])
    y_seq.append(y.iloc[i + sequence_length])

X_seq, y_seq = np.array(X_seq), np.array(y_seq)
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)


C:\Users\mague\AppData\Local\Temp\ipykernel_11916\1771548492.py:42: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk['time_stamp'] = pd.to_datetime(chunk['time_stamp'], errors='coerce')
C:\Users\mague\AppData\Local\Temp\ipykernel_11916\1771548492.py:42: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk['time_stamp'] = pd.to_datetime(chunk['time_stamp'], errors='coerce')
C:\Users\mague\AppData\Local\Temp\ipykernel_11916\1771548492.py:42: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  chunk['time_stamp'] = pd.to_datetime(chunk['time_stamp'], errors='coerce')
C:\Users\magu

In [4]:
# Modelo básico de LSTM
model_lstm_basic = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1, activation='sigmoid')
])

model_lstm_basic.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Entrenamiento
history_lstm_basic = model_lstm_basic.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# Evaluación
loss, accuracy = model_lstm_basic.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')


MemoryError: Unable to allocate 7.87 GiB for an array with shape (16240640, 10, 13) and data type float32

In [ ]:
# Modelo avanzado con GRU y Dropout
model_gru_advanced = Sequential([
    GRU(64, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.2),
    GRU(32),
    Dense(1, activation='sigmoid')
])

model_gru_advanced.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

# Entrenamiento
history_gru_advanced = model_gru_advanced.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# Evaluación
loss, accuracy = model_gru_advanced.evaluate(X_test, y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')


In [ ]:
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt

# Asumamos que dataTrain_timestamp['time_stamp'] y alguna métrica de compra recurrente ya están definidas
# Ejemplo de columna para análisis de series de tiempo
dataTrain_timestamp.set_index('time_stamp', inplace=True)
recurrent_series = dataTrain_timestamp['total_actions'].resample('M').sum()  # Ejemplo de resampleo mensual

# Visualizar la serie de tiempo
plt.figure(figsize=(10, 6))
plt.plot(recurrent_series, label='Total Actions Over Time')
plt.title('Series de Tiempo de Compras Recurrentes')
plt.xlabel('Fecha')
plt.ylabel('Acciones Totales')
plt.legend()
plt.show()

# Prueba de Dickey-Fuller para verificar estacionariedad
result = adfuller(recurrent_series.dropna())
print(f'Estadístico de ADF: {result[0]}')
print(f'p-valor: {result[1]}')


In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import warnings
warnings.filterwarnings("ignore")

# Configuración inicial para el modelo ARIMA
p, d, q = 1, 1, 1  # Valores iniciales

# Crear y ajustar el modelo ARIMA
model_arima_1 = ARIMA(recurrent_series, order=(p, d, q))
model_arima_1_fit = model_arima_1.fit()

# Resumen de resultados
print(model_arima_1_fit.summary())


In [ ]:
from pmdarima import auto_arima

# Ejecutar auto_arima para encontrar los mejores valores de p, d, q
auto_model = auto_arima(
    recurrent_series, 
    seasonal=False,  # Configuración sin estacionalidad, pero podemos probar con estacionalidad si es relevante
    stepwise=True, 
    trace=True
)

# Mejor modelo ARIMA sugerido por auto_arima
print(f"Mejor orden ARIMA sugerido: {auto_model.order}")


In [ ]:
# Predicciones para ambos modelos
recurrent_series_test = recurrent_series[-12:]  # Ejemplo de 12 meses para prueba

# Modelo ARIMA inicial
forecast_arima_1 = model_arima_1_fit.forecast(steps=12)
# Mejor modelo sugerido por auto_arima
forecast_auto_arima = auto_model.predict(n_periods=12)

# Graficar ambos resultados
plt.figure(figsize=(12, 6))
plt.plot(recurrent_series, label='Observado')
plt.plot(recurrent_series_test.index, forecast_arima_1, label='ARIMA Inicial', color='orange')
plt.plot(recurrent_series_test.index, forecast_auto_arima, label='Mejor ARIMA auto_arima', color='green')
plt.legend()
plt.title('Comparación de Modelos ARIMA')
plt.xlabel('Fecha')
plt.ylabel('Acciones Totales')
plt.show()
